In [1]:
import numpy as np
import pickle

In [2]:
with open("data/processed/spike_data.pickle", "rb") as f:
    spikes_data = pickle.load(f)

with open("data/processed/arm_data.pickle", "rb") as f:
    arm_data = pickle.load(f)

In [4]:
max_time = 1000
trials = 100
pre_movement = 300
neural_units = 98
directions = 8
axes = 3
t_gap = 20

In [5]:
training_data = np.zeros(
    (directions * trials * int(max_time / t_gap),
     3 * neural_units + 3)
)

In [6]:
# 40,000 observations
# 98 columns for firing rate, 98 columns for phase encoding,
# 98 columns for firing covariate, 3 columns for previous
# arm position
print(training_data.shape)

print(spikes_data.shape)
print(arm_data.shape)

(40000, 297)
(800, 98, 1000)
(800, 3, 1000)


In [7]:
time_knots = range(t_gap, max_time + t_gap, t_gap)

for idx, i in enumerate(time_knots):
    # First 98 columns of training_data describe firing rate
    training_data[
        directions * trials * idx:directions * trials * (1 + idx),
        :neural_units
    ] = np.mean(spikes_data[:, :, :i], axis=2)
    # Next 98 columns describe phase encoding
    training_data[
        directions * trials * idx:directions * trials * (1 + idx),
        neural_units:(2 * neural_units)
    ] = np.argmax(spikes_data[:, :, :i], axis=2)
    # Next 98 columns describe firing rate before movement
    training_data[
        directions * trials * idx:directions * trials * (1 + idx),
        (2 * neural_units):(3 * neural_units)
    ] = np.mean(spikes_data[:, :, :min(i, pre_movement)], axis=2)
    # Final 3 columns describe arm position at time knots
    training_data[
        directions * trials * idx:directions * trials * (1 + idx),
        (3 * neural_units):
    ] = arm_data[:, :, i-1]

training_data.shape

(40000, 297)

In [ ]:
# Add labels for classification

In [ ]:
# Change save to training_data
# out_file = "data/processed/neural_feats.pickle"
# with open(out_file, "wb") as f:
#     pickle.dump(neural_feats, f)